In [23]:
import torch
import sys, os, shutil, io, base64
from pathlib import Path
from PIL import Image as PILImage
from transformers import CLIPProcessor, CLIPModel
sys.path.append(str(Path('impl.ipynb').resolve().parents[0]))

CLIP_PATH = '../data/models/CLIP'
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
processor = CLIPProcessor.from_pretrained(CLIP_PATH)
model = CLIPModel.from_pretrained(CLIP_PATH)
model.to(device)

def get_image_embedding(img_type='path', image_path:str="", bs64_image:str=""):
    model.eval()
    if img_type == 'path':
        image = PILImage.open(image_path).convert("RGB")
    elif img_type == 'base64':
        image = PILImage.open(io.BytesIO(base64.b64decode(bs64_image))).convert("RGB")
    else:
        raise ValueError(f"Unsupported type: {img_type}")

    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        embedding = model.get_image_features(**inputs)
    return embedding.squeeze().tolist()

def get_text_embedding(txt):
    model.eval()
    inputs = processor(text=txt, return_tensors="pt").to(device)
    with torch.no_grad():
        embedding = model.get_text_features(**inputs)
    return embedding.squeeze().tolist()

### If need downloading AITeamVN's model, run the following code

In [ ]:
# import torch
# from sentence_transformers import SentenceTransformer
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

# # Download from Hugging Face and cache locally
# saving_model = SentenceTransformer("AITeamVN/Vietnamese_Embedding_v2")
# saving_model.max_seq_length = 2048
# saving_model = saving_model.to(device)

# # Save it to your desired folder (e.g., ./local_models/vietnamese_embedding)
# save_path = "../data/models/AITeamVN"
# saving_model.save(save_path)

# print(f"Model saved to {save_path}")


### Load AITeamVN model from local

In [3]:
import torch
from sentence_transformers import SentenceTransformer
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")

# Load from local folder
model_path = "../data/models/AITeamVN"
AITeamVNModel = SentenceTransformer(model_path)
AITeamVNModel.max_seq_length = 2048
AITeamVNModel = AITeamVNModel.to(device)

def get_aiteamvn_text_emb(text_inputs):
    embeddings = AITeamVNModel.encode(
        text_inputs,
        device=device, 
        batch_size=8,
        convert_to_tensor=True,
        show_progress_bar=False
    )
    return embeddings.tolist()

### Qdrant and SQL definition

In [4]:
from postgre_conn import PostgresDB
from qdrant.qdrant_store import QdrantVectorStore
from qdrant_client.http import models
from qdrant_client.http.models import Filter, FieldCondition, Match, MatchText, MatchValue

PASSWORD="PixelFlow"
PGDB = PostgresDB(PASSWORD)

COLLECTION_NAME = "FrameClipEmbedding"
QVS = QdrantVectorStore(
    url="http://34.63.214.230:6333",
    collection_name=COLLECTION_NAME,
    vector_sizes={"image": 512, "text": 1024}
)
client = QVS.client
collections = client.get_collections()
print("Collections:", collections)

print("✅ Connected and collection ready!")


2025-08-29 21:55:37.514 | INFO     | qdrant.qdrant_store:_create_collection:22 - Collection 'FrameClipEmbedding' already exists. Skip creation.


PostgreSQL connection established.
Collections: collections=[CollectionDescription(name='FrameBeitEmbedding'), CollectionDescription(name='FrameClipEmbedding')]
✅ Connected and collection ready!


### Gemini API Loading

In [5]:
import os
from geminiapi import generate_text, generate_image, display_base64_image
from dotenv import load_dotenv; load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GEMINI_MODEL = os.getenv("GEMINI_MODEL", "gemini-2.5-flash")

In [6]:
def translate_text(text, target_language:str="english"):
    input_prompt = f"Translate the following text to {target_language}: {text}"
    translated_text = generate_text(input_prompt, api_key=GEMINI_API_KEY, model=GEMINI_MODEL)
    return translated_text

In [ ]:
input_prompt = "Explain the concept of quantum computing in simple terms."
text_response = generate_text(input_prompt, api_key=GEMINI_API_KEY, model=GEMINI_MODEL)
print(text_response)

In [ ]:
image_prompt = "A scenic landscape with a cute cat on the mountain" 
save_directory = "generated_images"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

base64_image, generated_text = generate_image(
                                            api_key=GEMINI_API_KEY,
                                            image_prompt=image_prompt,
                                            save_image=False,
                                            save_path=save_directory,
                                            target_width=1024,
                                            target_height=768,
                                            resize_image_flag=True,
                                        )

if base64_image:
    display_base64_image(base64_image)

## Query

KIS
1. Cảnh quay bằng flycam một cây cầu ở TP Hồ Chí Minh, tiếp theo đến cảnh quay tòa nhà Bitexco. Một vài cảnh sau đó chuyển qua quay hình ảnh hồ gươm tại Hà Nội.
2. Một người đàn ông đang trả lời phỏng vấn trong một lễ hội. Phía sau người đàn ông này là một vật trang trí có hình dáng con chim màu tím.

QA
1. Đây là phần giới thiệu phần thưởng cho một cuộc thi được thiết kế với nền xanh, chữ trắng phủ lên video giới thiệu cuộc thi. Tổng có 18 giải thưởng chính cho cuộc thi. Hỏi tổng giá trị của các giải thưởng chính là bao nhiêu?
2. Một bác sĩ tóc bạc, đeo kính đang trả lời phỏng vấn. Sau đó là cảnh vị bác sĩ này đang nói chuyện với một bệnh nhân nữ quay lưng lại với camera mặc áo đen. Trên bàn có 2 cây bút. Bên trái áo của bác sĩ có thêu dòng chữ màu đỏ gồm tên, học hàm, học vị của vị bác sĩ này. Hỏi họ và tên đầy đủ của bác sĩ là gì?

TRAKE-1
- E1: Người đầu bếp cho cá vào một tô màu trắng. Hãy lấy khoảnh khắc con cá cuoói cùng rớt khỏi dĩa
- E2: Người đầu bếp đổ bột vào một tô cá để chiên. Hãy lấy khoảnh khắc đầu tiên mà chiếc đũa của người đầu bếp chạm vào cá để trộn bột.
- E3: Tiếp theo, người đầu bếp này dùng đũa để kiểm tra độ nóng của dầu. Hãy lấy khoảnh khắc đầu tiên chiếc đũa được nhấc ra khỏi dầu.

TRAKE-2
- E1: Một người  đang cắt đôi ổ bánh mì có rắc mè rồi đem nướng trên chảo. Hãy lấy khoảnh khắc chiếc dao cắt qua hoàn toàn chiếc bánh.
- E2: Sau đó người này rắc bột lên những miếng thịt, trong quá trình này người đầu bếp lật những miếng thịt để rắc bột đều hai mặt. Hãy lấy khoảnh khắc đầu tiên người đầu bếp này buông tay khỏi miếng thịt sau khi lật miếng thịt đầu tiên. 
- E3: Các miếng thịt sau đó được đem đi áp chảo cùng với bơ (3 ngang 1 dọc theo chiều của camera). Hãy lấy khoảnh khắc đầu tiên người đầu bếp cầm vào chảo để nhấc lên đảo bơ đều xung quanh.


### 1. Query Text to Image 

In [7]:
q1 = "Cảnh quay bằng flycam một cây cầu ở TP Hồ Chí Minh, tiếp theo đến cảnh quay tòa nhà Bitexco. Một vài cảnh sau đó chuyển qua quay hình ảnh hồ gươm tại Hà Nội."
q2 = "Một người đàn ông đang trả lời phỏng vấn trong một lễ hội. Phía sau người đàn ông này là một vật trang trí có hình dáng con chim màu tím."

q1_eng = translate_text(q1, target_language="english")
q2_eng = translate_text(q2, target_language="english")

print(q2_eng)
clip_txt_emb = get_text_embedding(q2_eng)

from IPython.display import Image, display
points = client.query_points(
            collection_name="FrameClipEmbedding",
            query=clip_txt_emb,
            using='image',
            limit=5,
            with_payload=True
        ).points

for point in points:
    frame_id = point.id
    frame = PGDB.fetch_frame_by_id(frame_id)
    frame_url = frame[4]

    # Display images
    display(Image(url=frame_url))
    
    print(frame_url)
    print(point.payload)
    print('-'*50)

A man is being interviewed at a festival. Behind the man is a purple bird-shaped decoration.



https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V012/160.webp
{'video_id': '06ab05d4-5c8d-41e8-a478-b7471e1bcdb3', 'video_name': 'L22_V012', 'file_index': 160, 'objects': [{'object_type': 'person', 'count': 2}, {'object_type': 'handbag', 'count': 1}, {'object_type': 'bird', 'count': 1}], 'ocr': 'HTV7 HD, Dự thảo Luật sửa đổi, bổ sung một', 'description': 'Hình ảnh chụp một người đàn ông mặc áo sơ mi trắng, đội mũ và đeo túi xách đang đứng trước một gian hàng trang trí với nhiều hoa tươi và mô hình đồ họa.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L24/L24_V012/013.webp
{'video_id': 'f82aafbb-35a3-494d-8b49-06fb0f76a87d', 'video_name': 'L24_V012', 'file_index': 13, 'objects': [{'object_type': 'person', 'count': 12}], 'ocr': 'HTV, Bí quyết thành công truyền hình', 'description': 'Người đàn ông trong ảnh đang được một phóng viên quay phim, anh ấy mặc áo thun trắng có in chữ Sáng màu vàng và đeo vòng cổ bằng kim loại.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V010/095.webp
{'video_id': '314e42e6-c6a4-457c-9ba4-a434d97aad61', 'video_name': 'L22_V010', 'file_index': 95, 'objects': [{'object_type': 'person', 'count': 1}], 'ocr': 'Ba Lan đặt mua hàng trăm tên lửa không đối không của Mỹ', 'description': 'Người đàn ông trong ảnh đang nói chuyện trước một cửa hàng bày bán đồ trang trí Tết.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V011/202.webp
{'video_id': '896b98ee-25ba-43bc-9734-e9bd3c3e52a8', 'video_name': 'L22_V011', 'file_index': 202, 'objects': [{'object_type': 'cell phone', 'count': 1}, {'object_type': 'person', 'count': 3}, {'object_type': 'umbrella', 'count': 1}], 'ocr': 'HTV7 HD, 18:43:15, Ngân hàng Nhật Bản khai thác năng', 'description': 'Hình ảnh cho thấy một người phụ nữ đang sử dụng điện thoại di động để chụp ảnh hoặc quay video cảnh quan hoa nhiều màu sắc trước một tòa nhà lớn.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L24/L24_V012/012.webp
{'video_id': 'f82aafbb-35a3-494d-8b49-06fb0f76a87d', 'video_name': 'L24_V012', 'file_index': 12, 'objects': [{'object_type': 'person', 'count': 5}, {'object_type': 'motorcycle', 'count': 1}, {'object_type': 'frisbee', 'count': 1}], 'ocr': 'HTV, Báo chí Trung ương Việt Nam', 'description': 'Người đàn ông trong ảnh đang được một phóng viên quay phim, anh ấy mặc áo thun trắng có in chữ màu vàng. Phía sau anh ta là hai con sư tử múa lân.'}
--------------------------------------------------


In [10]:
q11 = "Cảnh quay bằng flycam một cây cầu ở TP Hồ Chí Minh"
q12 = "Cảnh quay tòa nhà Bitexco"
q13 = "Hình ảnh hồ gươm tại Hà Nội"

q11_eng = translate_text(q11, target_language="english")
print(q11_eng)
clip_txt_emb = get_text_embedding(q11_eng)

from IPython.display import Image, display
points = client.query_points(
            collection_name="FrameClipEmbedding",
            query=clip_txt_emb,
            using='image',
            limit=5,
            with_payload=True
        ).points

for point in points:
    frame_id = point.id
    frame = PGDB.fetch_frame_by_id(frame_id)
    frame_url = frame[4]

    # Display images
    display(Image(url=frame_url))
    
    print(frame_url)
    print(point.payload)
    print('-'*50)

Flycam footage of a bridge in Ho Chi Minh City.



https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V003/094.webp
{'video_id': '52c7f7fe-291b-444f-980e-6e204e14b38e', 'video_name': 'L22_V003', 'file_index': 94, 'objects': [{'object_type': 'car', 'count': 29}, {'object_type': 'truck', 'count': 1}], 'ocr': 'HTV7 HD, Triển lãm “Duyên” của họa sĩ Lê Thiết', 'description': 'Ảnh chụp cảnh đường phố và cầu bắc sông ở Bình Định vào buổi tối, với nhiều phương tiện giao thông lưu thông. Có một cột mốc lớn được trang trí công phu.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V023/101.webp
{'video_id': '2258caeb-aa4c-4294-a92b-2c7d35113407', 'video_name': 'L22_V023', 'file_index': 101, 'objects': [{'object_type': 'car', 'count': 21}, {'object_type': 'bus', 'count': 1}, {'object_type': 'truck', 'count': 3}, {'object_type': 'boat', 'count': 1}], 'ocr': 'HTV7 HD, 18:36:23, Palestine cảnh báo Israel', 'description': 'Hình ảnh chụp từ trên cao một cây cầu dây văng bắc qua sông với nhiều phương tiện giao thông lưu thông. Phía dưới cầu là khu vực dân cư và các công trình xây dựng.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V031/045.webp
{'video_id': '226feb1b-f407-43a6-80c4-267e7b8790f2', 'video_name': 'L22_V031', 'file_index': 45, 'objects': [{'object_type': 'boat', 'count': 17}, {'object_type': 'truck', 'count': 2}, {'object_type': 'car', 'count': 3}, {'object_type': 'bus', 'count': 1}], 'ocr': '', 'description': 'Ảnh chụp từ trên cao một cây cầu dây văng bắc qua sông, với nhiều phương tiện giao thông đang lưu thông. Phía dưới cùng của ảnh có dòng chữ Liên hoan Ca Múa Nhạc toàn quốc - 2024 và logo HTV7 HD.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L23/L23_V021/118.webp
{'video_id': '1cbbac26-1542-47c5-a781-93f433ff91f1', 'video_name': 'L23_V021', 'file_index': 118, 'objects': [{'object_type': 'car', 'count': 6}, {'object_type': 'boat', 'count': 9}, {'object_type': 'sink', 'count': 1}, {'object_type': 'person', 'count': 1}, {'object_type': 'airplane', 'count': 1}, {'object_type': 'truck', 'count': 1}], 'ocr': 'REPLAY, HTV HẾ THAO', 'description': 'Hình ảnh chụp từ trên cao một cây cầu bắc qua sông với nhiều phương tiện giao thông lưu thông. Cây cầu có thiết kế hiện đại và rộng rãi.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L28/L28_V013/161.webp
{'video_id': '37da7388-54f3-49c1-b170-7e0846153ad2', 'video_name': 'L28_V013', 'file_index': 161, 'objects': [{'object_type': 'car', 'count': 7}, {'object_type': 'truck', 'count': 2}], 'ocr': '', 'description': 'Ảnh chụp từ trên cao cho thấy một con sông rộng lớn với nhiều nhà cửa ven bờ. Có một cây cầu bắc qua sông, và có thể nhìn thấy một vài chiếc thuyền nhỏ trên sông. Văn bản duy nhất trong ảnh là HTV Online ở góc trên bên trái.'}
--------------------------------------------------


In [17]:
q11 = "Cảnh quay bằng flycam một cây cầu ở TP Hồ Chí Minh"
q12 = "Cảnh quay tòa nhà Bitexco"
q13 = "Hình ảnh hồ gươm tại Hà Nội"

q12_eng = translate_text(q12, target_language="english")
print(q12_eng)
clip_txt_emb = get_text_embedding(q12_eng)

from IPython.display import Image, display
points = client.query_points(
            collection_name="FrameClipEmbedding",
            query=clip_txt_emb,
            using='image',
            limit=5,
            with_payload=True
        ).points

for point in points:
    frame_id = point.id
    frame = PGDB.fetch_frame_by_id(frame_id)
    frame_url = frame[4]

    # Display images
    display(Image(url=frame_url))
    
    print(frame_url)
    print(point.payload)
    print('-'*50)

The Bitexco Financial Tower scene/shot.



https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V016/040.webp
{'video_id': 'a225319c-2e2e-49f0-b300-9e6d291e1dd9', 'video_name': 'L22_V016', 'file_index': 40, 'objects': [{'object_type': 'boat', 'count': 9}], 'ocr': 'HTV7 HD, 18:31:54, TP.HCM: Công an huyện Bình Chánh tăng cường tổ công tác kiểm soát nồng độ cồn', 'description': 'Ảnh chụp toàn cảnh thành phố Hồ Chí Minh với nhiều tòa nhà cao tầng, bao gồm một tòa nhà tháp độc đáo ở trung tâm.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L21/L21_V016/008.webp
{'video_id': 'a41b84bb-5601-4df3-b027-591764f6431f', 'video_name': 'L21_V016', 'file_index': 8, 'objects': [{'object_type': 'truck', 'count': 2}], 'ocr': 'HTV9, 06:30:29 HD', 'description': 'Ảnh chụp từ trên cao một thành phố đông đúc với nhiều tòa nhà cao tầng, bờ sông và cây xanh.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L22/L22_V016/312.webp
{'video_id': 'a225319c-2e2e-49f0-b300-9e6d291e1dd9', 'video_name': 'L22_V016', 'file_index': 312, 'objects': [{'object_type': 'person', 'count': 3}, {'object_type': 'car', 'count': 8}, {'object_type': 'traffic light', 'count': 4}], 'ocr': 'HTV7 HD, 18:51:36, Nhật Bản thúc đẩy cải tạo đất trống thành không gian công cộng', 'description': 'Hình ảnh chụp một tòa nhà cao tầng hiện đại ở Nhật Bản, với các chi tiết kiến trúc và cảnh quan xung quanh.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L21/L21_V017/037.webp
{'video_id': '6548757a-3613-4598-9083-03abe900ed87', 'video_name': 'L21_V017', 'file_index': 37, 'objects': [{'object_type': 'car', 'count': 2}, {'object_type': 'bus', 'count': 1}], 'ocr': 'Nghệ An: Hơn 750ha rừng thông bị sâu róm tàn phá, có nguy cơ lan rộng', 'description': 'Ảnh chụp toàn cảnh thành phố Hồ Chí Minh với nhiều tòa nhà cao tầng và cây xanh, cho thấy sự phát triển đô thị mạnh mẽ.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L23/L23_V006/088.webp
{'video_id': '30b55dc3-14f4-46b3-94e4-f212fac30545', 'video_name': 'L23_V006', 'file_index': 88, 'objects': [{'object_type': 'car', 'count': 14}, {'object_type': 'clock', 'count': 1}, {'object_type': 'truck', 'count': 1}, {'object_type': 'person', 'count': 13}], 'ocr': 'CHĂNG 6, 02:46:45, 00:26, HTV HẾ THAO', 'description': 'Ảnh chụp cảnh đường phố đông đúc với nhiều người đang tham gia một cuộc biểu tình hoặc sự kiện nào đó. Có thể thấy các tòa nhà, cây xanh và xe cộ trên đường.'}
--------------------------------------------------


In [11]:
q11 = "Cảnh quay bằng flycam một cây cầu ở TP Hồ Chí Minh"
q12 = "Cảnh quay tòa nhà Bitexco"
q13 = "Hình ảnh hồ gươm tại Hà Nội"

q13_eng = translate_text(q13, target_language="english")
print(q13_eng)
clip_txt_emb = get_text_embedding(q13_eng)

from IPython.display import Image, display
points = client.query_points(
            collection_name="FrameClipEmbedding",
            query=clip_txt_emb,
            using='image',
            limit=5,
            with_payload=True
        ).points

for point in points:
    frame_id = point.id
    frame = PGDB.fetch_frame_by_id(frame_id)
    frame_url = frame[4]

    # Display images
    display(Image(url=frame_url))
    
    print(frame_url)
    print(point.payload)
    print('-'*50)

The image of Hoan Kiem Lake in Hanoi.



https://storage.googleapis.com/hcmai25/Keyframes_L28/L28_V013/160.webp
{'video_id': '37da7388-54f3-49c1-b170-7e0846153ad2', 'video_name': 'L28_V013', 'file_index': 160, 'objects': [{'object_type': 'person', 'count': 3}, {'object_type': 'bench', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh chụp một con sông với hàng cây xanh mát bên bờ. Có thể nhìn thấy một số tòa nhà ở phía xa.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L29/L29_V007/153.webp
{'video_id': 'a45923bb-06bb-48fa-a4b4-4fd8c885209d', 'video_name': 'L29_V007', 'file_index': 153, 'objects': [], 'ocr': 'HTV online', 'description': 'Hình ảnh chụp một tòa tháp kiến trúc truyền thống ở hồ nước, với lá cờ đỏ sao vàng bay phấp phới trên đỉnh.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L28/L28_V002/322.webp
{'video_id': '5036eb7f-65dd-4651-8a7e-9609bb03c67a', 'video_name': 'L28_V002', 'file_index': 322, 'objects': [{'object_type': 'person', 'count': 20}, {'object_type': 'motorcycle', 'count': 12}, {'object_type': 'bicycle', 'count': 4}, {'object_type': 'bench', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Ảnh chụp một con đường rộng với nhiều cây xanh và trang trí bằng đèn lồng màu vàng, hoa hướng dương và cờ đỏ sao vàng.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L25/L25_V024/160.webp
{'video_id': 'a429616a-bdb3-4b96-a958-c0baf8c12e59', 'video_name': 'L25_V024', 'file_index': 160, 'objects': [{'object_type': 'person', 'count': 15}, {'object_type': 'tie', 'count': 1}, {'object_type': 'chair', 'count': 1}], 'ocr': 'ĐỒNG BẰNG SÔNG HỒNG, CHUYỂN DỊCH CƠ CẤP KINH TẾ, Các định hướng chính, Đối với khu vực III, Du lịch là một ngành tiềm năng, Tài chính, ngân hàng, giáo dục - đào tạo...', 'description': 'Ảnh hiển thị một bài giảng về địa lý các vùng kinh tế Việt Nam, tập trung vào chủ đề Đồng bằng sông Hồng và chuyển dịch cơ cấu kinh tế.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L28/L28_V002/323.webp
{'video_id': '5036eb7f-65dd-4651-8a7e-9609bb03c67a', 'video_name': 'L28_V002', 'file_index': 323, 'objects': [{'object_type': 'person', 'count': 16}, {'object_type': 'motorcycle', 'count': 11}, {'object_type': 'bicycle', 'count': 3}], 'ocr': 'HTV Online', 'description': 'Ảnh chụp một con đường với nhiều người đi xe máy, được trang trí bằng đèn lồng và hoa mai. Có cây lớn ở hai bên đường.'}
--------------------------------------------------


In [14]:
q3 = "Một người đang cắt đôi ổ bánh mì có rắc mè rồi đem nướng trên chảo"

q3_eng = translate_text(q3, target_language="english")
print(q3_eng)
clip_txt_emb = get_text_embedding(q3_eng)

from IPython.display import Image, display
points = client.query_points(
            collection_name="FrameClipEmbedding",
            query=clip_txt_emb,
            using='image',
            limit=5,
            with_payload=True
        ).points

for point in points:
    frame_id = point.id
    frame = PGDB.fetch_frame_by_id(frame_id)
    frame_url = frame[4]

    # Display images
    display(Image(url=frame_url))
    
    print(frame_url)
    print(point.payload)
    print('-'*50)

A person is slicing a sesame seed bun in half and toasting it on a pan.



https://storage.googleapis.com/hcmai25/Keyframes_L26_e/L26_V406/048.webp
{'video_id': '06a9adee-43c8-45e3-998e-27ab754dab61', 'video_name': 'L26_V406', 'file_index': 48, 'objects': [{'object_type': 'person', 'count': 1}, {'object_type': 'cake', 'count': 1}, {'object_type': 'dining table', 'count': 1}, {'object_type': 'oven', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một chiếc bánh mì được phủ rắc vừng trên một chiếc khay phẳng màu đen. Một bàn tay đang đổ bột lên bề mặt bánh mì.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_e/L26_V406/049.webp
{'video_id': '06a9adee-43c8-45e3-998e-27ab754dab61', 'video_name': 'L26_V406', 'file_index': 49, 'objects': [{'object_type': 'oven', 'count': 1}, {'object_type': 'cake', 'count': 2}, {'object_type': 'donut', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hai chiếc bánh mì được đặt trên một khay trộn bằng kim loại trên bếp gas.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_d/L26_V347/059.webp
{'video_id': '7de3938f-a5f7-41a2-845a-32f7b407f117', 'video_name': 'L26_V347', 'file_index': 59, 'objects': [{'object_type': 'person', 'count': 3}, {'object_type': 'bowl', 'count': 1}, {'object_type': 'cake', 'count': 1}, {'object_type': 'dining table', 'count': 2}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một người mặc áo trắng đang sử dụng một chiếc đĩa thủy tinh để pha chế một loại bột màu nâu nhạt, có thể là bột gạo hoặc bột ngũ cốc.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_d/L26_V377/126.webp
{'video_id': '0e5a6636-79af-476e-8423-c4e172699184', 'video_name': 'L26_V377', 'file_index': 126, 'objects': [{'object_type': 'person', 'count': 2}, {'object_type': 'book', 'count': 1}, {'object_type': 'knife', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một người đang sử dụng một chiếc chảo nhỏ màu đen đặt trên bếp điện. Người này đang cầm một vật thể giống như bánh mì hoặc bột sợi và dán nó lên bề mặt của chảo.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_e/L26_V406/047.webp
{'video_id': '06a9adee-43c8-45e3-998e-27ab754dab61', 'video_name': 'L26_V406', 'file_index': 47, 'objects': [{'object_type': 'cake', 'count': 1}, {'object_type': 'oven', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh chụp một chiếc bánh mì giòn rụm được đặt trong một khay đen.'}
--------------------------------------------------


### 1. Query Image to Image 

In [ ]:
q2 = "Cảnh tòa nhà Bitexco ở TP Hồ Chí Minh"

# Gen image
q2_eng = translate_text(q2, target_language="english")
bs64_img, _ = generate_image(api_key=GEMINI_API_KEY, image_prompt=q2_eng)
# Visualize base64 image
img = PILImage.open(io.BytesIO(base64.b64decode(bs64_img))).convert("RGB")
display(img)

img_emb = get_image_embedding(bs64_image=bs64_img, img_type='base64')

from IPython.display import Image, display
points = client.query_points(
            collection_name="FrameClipEmbedding",
            query=img_emb,
            using='image',
            limit=5,
            with_payload=True
        ).points

for point in points:
    frame_id = point.id
    frame = PGDB.fetch_frame_by_id(frame_id)
    frame_url = frame[4]

    # Display images
    display(Image(url=frame_url))
    
    print(frame_url)
    print(point.payload)
    print('-'*50)

In [ ]:
q2 = "rắc bột lên những miếng thịt"

"""
- E1: Người đầu bếp cho cá vào một tô màu trắng. Hãy lấy khoảnh khắc con cá cuoói cùng rớt khỏi dĩa
- E2: Người đầu bếp đổ bột vào một tô cá để chiên. Hãy lấy khoảnh khắc đầu tiên mà chiếc đũa của người đầu bếp chạm vào cá để trộn bột.
- E3: Tiếp theo, người đầu bếp này dùng đũa để kiểm tra độ nóng của dầu. Hãy lấy khoảnh khắc đầu tiên chiếc đũa được nhấc ra khỏi dầu.

TRAKE-2
- E1: Một người  đang cắt đôi ổ bánh mì có rắc mè rồi đem nướng trên chảo. Hãy lấy khoảnh khắc chiếc dao cắt qua hoàn toàn chiếc bánh.
- E2: Sau đó người này rắc bột lên những miếng thịt, trong quá trình này người đầu bếp lật những miếng thịt để rắc bột đều hai mặt. Hãy lấy khoảnh khắc đầu tiên người đầu bếp này buông tay khỏi miếng thịt sau khi lật miếng thịt đầu tiên.
"""
aiteamvn_emb = get_aiteamvn_text_emb([q2])

from IPython.display import Image, display
points = client.query_points(
            collection_name="FrameClipEmbedding",
            query=aiteamvn_emb,
            using='text',
            limit=10,
            with_payload=True
        ).points

for point in points:
    frame_id = point.id
    frame = PGDB.fetch_frame_by_id(frame_id)
    frame_url = frame[4]

    # Display images
    display(Image(url=frame_url))
    
    print(frame_url)
    print(point.payload)
    print('-'*50)

https://storage.googleapis.com/hcmai25/Keyframes_L26_a/L26_V027/057.webp
{'video_id': 'd3f47a58-204c-4427-9227-155eefd73025', 'video_name': 'L26_V027', 'file_index': 57, 'objects': [{'object_type': 'spoon', 'count': 1}, {'object_type': 'bowl', 'count': 1}, {'object_type': 'person', 'count': 4}, {'object_type': 'dining table', 'count': 1}, {'object_type': 'cup', 'count': 1}, {'object_type': 'banana', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một người đang rắc bột lên các miếng thịt đã được nấu chín.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_b/L26_V199/042.webp
{'video_id': '38e1a086-f7af-470e-b41e-f500cb662c4e', 'video_name': 'L26_V199', 'file_index': 42, 'objects': [{'object_type': 'bowl', 'count': 6}, {'object_type': 'spoon', 'count': 1}, {'object_type': 'person', 'count': 3}, {'object_type': 'dining table', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một người đang rắc bột trên thịt đã được cắt lát.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_d/L26_V371/095.webp
{'video_id': '050227c4-d907-4c7d-a7c3-88e13c891709', 'video_name': 'L26_V371', 'file_index': 95, 'objects': [{'object_type': 'bowl', 'count': 2}, {'object_type': 'person', 'count': 1}, {'object_type': 'dining table', 'count': 1}, {'object_type': 'donut', 'count': 2}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một người đang rắc bột năng lên các miếng thịt đã được chế biến sẵn.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_c/L26_V244/070.webp
{'video_id': 'fdadf386-5080-40f8-9303-4aa8a3bc7efc', 'video_name': 'L26_V244', 'file_index': 70, 'objects': [{'object_type': 'person', 'count': 1}, {'object_type': 'bowl', 'count': 1}, {'object_type': 'cake', 'count': 1}], 'ocr': '', 'description': 'Ảnh cho thấy một người đang rắc bột Ragu Bột 1M lên một số miếng thịt đã được chế biến sẵn.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_c/L26_V283/082.webp
{'video_id': 'c3b110ee-9bcb-4efe-a1a0-d3dec0eb25e0', 'video_name': 'L26_V283', 'file_index': 82, 'objects': [{'object_type': 'spoon', 'count': 1}, {'object_type': 'person', 'count': 1}, {'object_type': 'bowl', 'count': 2}], 'ocr': '', 'description': 'Hình ảnh cho thấy một người đang rắc bột xào vào một tô thịt đã được nấu chín. Bột xào có màu vàng nhạt và được phủ đều trên bề mặt thịt.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_b/L26_V188/053.webp
{'video_id': '9aa88d27-aa95-4ca4-be1d-afc584704064', 'video_name': 'L26_V188', 'file_index': 53, 'objects': [{'object_type': 'person', 'count': 1}, {'object_type': 'bowl', 'count': 2}, {'object_type': 'cup', 'count': 1}, {'object_type': 'cake', 'count': 5}, {'object_type': 'donut', 'count': 1}, {'object_type': 'dining table', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một bàn tay đang rắc bột lên các miếng thịt đã được cắt sẵn trong một chiếc bát thủy tinh trong suốt.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_b/L26_V129/096.webp
{'video_id': '4b7eba73-c2ed-4bb0-90aa-d925e935489b', 'video_name': 'L26_V129', 'file_index': 96, 'objects': [{'object_type': 'person', 'count': 2}, {'object_type': 'bowl', 'count': 2}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một người đang chuẩn bị món ăn, cụ thể là việc rắc bột gạo lên các miếng thịt đã được nướng trong chảo.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_b/L26_V188/052.webp
{'video_id': '9aa88d27-aa95-4ca4-be1d-afc584704064', 'video_name': 'L26_V188', 'file_index': 52, 'objects': [{'object_type': 'bowl', 'count': 1}, {'object_type': 'person', 'count': 1}, {'object_type': 'cake', 'count': 2}, {'object_type': 'dining table', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một bàn tay đang rắc bột màu nâu nhạt lên một số miếng thịt đã được cắt thành từng lát mỏng, đặt trong một chiếc bát thủy tinh trong suốt.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_c/L26_V208/072.webp
{'video_id': 'c3291f07-30de-41ca-919a-572b06f8b5f3', 'video_name': 'L26_V208', 'file_index': 72, 'objects': [{'object_type': 'spoon', 'count': 3}, {'object_type': 'bowl', 'count': 1}, {'object_type': 'fork', 'count': 1}, {'object_type': 'person', 'count': 1}, {'object_type': 'knife', 'count': 1}, {'object_type': 'cell phone', 'count': 1}, {'object_type': 'dining table', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một người đang dùng đũa và thìa để rắc bột xốp lên một loại thịt đã được nấu chín.'}
--------------------------------------------------


https://storage.googleapis.com/hcmai25/Keyframes_L26_c/L26_V235/034.webp
{'video_id': 'b24aaa75-601a-4ec3-ade8-944e8b4d610c', 'video_name': 'L26_V235', 'file_index': 34, 'objects': [{'object_type': 'dining table', 'count': 1}, {'object_type': 'bowl', 'count': 1}], 'ocr': 'HTV Online', 'description': 'Hình ảnh cho thấy một số miếng thịt đã được cắt thành hình vuông, đặt trên một chiếc đĩa gỗ. Trên mỗi miếng thịt đều có lớp bột dừa trắng phủ lên.'}
--------------------------------------------------
